# Geos-Chem analysis

Este notebook possibilita realizar a análise da saída do Geos-Chem. Ele conta com instruções para:

- Abertura do arquivo de saída em formatos `netcdf` e `bpch`
- Leitura das variáveis, atributos e coordenadas
- Exibição da concentração do elemento químico com e sem cartografia
- Exibição interativa com seleção da variável e lev

Importação das bibliotecas:

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

from ipywidgets import interact, widgets
from matplotlib.animation import FuncAnimation, writers
from gamap_colormap import WhGrYlRd
from IPython.display import HTML
from math import ceil

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import warnings
import xbpch
import glob

warnings.filterwarnings("ignore")  # hide some xbpch warnings

# Set up formatting for the movie files
Writer = writers['ffmpeg']
writer = Writer(fps=1, metadata=dict(artist='Me'), bitrate=1800)

## Global variables

Caminhos para os arquivos:

- `trac`: arquivo de saída do Geos-Chem, normalmente em formato bpch our netcdf
- `tracerinfo` e `diaginfo`: caso `trac` esteja em formato bpch, os arquivos [tracerinfo|diaginfo].dat são necessários para leitura
- `f_out`: caminho para salvar as figuras do plot, caso seja habilitada nas funções

Se a análise for em múltiplos arquivos bpch da saída do ND49. Basta colocar o path da pasta em `trac`.

In [3]:
# Files path
ARGS = {
    "trac": "/home/user/dev/my/geoschem-data-analysis/temp/history/GC_201501-02_ND49_1_2_4/",
    "tracerinfo": "/home/user/dev/my/geoschem-data-analysis/temp/history/GC_201501-02_ND49_1_2_4/tracerinfo.dat",
    "diaginfo": "/home/user/dev/my/geoschem-data-analysis/temp/history/GC_201501-02_ND49_1_2_4/diaginfo.dat",
    "f_out": lambda x, l, t: f"out/img/{x}_lev_{l}_{t}.png",
    "anim": 'out/output_geoschem.mp4',
}

## Functions

Função para retornar um array com elementos de `0...(variable - 1)`

_Exemplo:_

- variable = 2 -> rlen(variable) = [0, 1]
- variable = 5 -> rlen(variable) = [0, 1, 2, 3, 4]

In [4]:
def rlen(variable):
    return range(len(variable))

Função para exibir os dados por plot em mapa cartográfico.

- `emiss`: xarray com os dados de saída do Geos-Chem
- `tracers`: uma lista das variáveis que deseja exibir
- `times`: uma lista de inteiros com os índices dos times da simulação
- `levs`: uma lista de inteiros com as `lev` que deseja exibir
- `save`: um boleano indicando se deseja salvar a(s) imagem(es) de output

In [5]:
def plot_static(
    emiss: xr.Dataset,
    tracers: [str],
    times: [int],
    levs: [int],
    save: bool = False
):
    emissions = emiss[tracers]
    full_time = emissions.time.values
    time_size = len(times)
    # Row and col dynamic processing for subplots
    rows = ceil(time_size/2)
    cols = 1 if time_size == 1 else 2
    
    # Get each tracer in emissions xarray
    for tracer in emissions.values():
        print(f"TRACER {tracer.name}")
        
        # Check if exists lev on tracer
        have_lev = True
        identified_levs = levs
        
        if "lev" not in tracer.dims:
            have_lev = False
            identified_levs = ["the is no lev"]

        # Get each lev
        for l in identified_levs:
            print(f"LEV {l}")
            
            # Create the subplots
            fig, axes = plt.subplots(
                rows,
                cols,
                figsize=(15, 7),
                subplot_kw={"projection": ccrs.PlateCarree()}
            )
            
            # Prepare `isel` parameters
            kwargs = {}
            if have_lev:
                kwargs["lev"] = l

            # Plot diagnotics for each time
            for ax, t in zip(axes, times):
                kwargs["time"] = t # adds time in parameters
                tracer.isel(**kwargs).plot(
                    ax=ax,
                    cmap=WhGrYlRd,
                    vmin=0,
                    cbar_kwargs={"shrink": 0.5, "label": tracer.units},
                )
                ax.set_title(f"{tracer.name} lev {l} - {full_time[t]}")
                ax.coastlines()
                ax.gridlines(linestyle="--")
            
            plt.show()
            plt.tight_layout()
                
            if save:
                plt.savefig(ARGS["f_out"](tracer.name, l, full_time[t]))

## Analysis

Leitura do arquivo `bpch` ou `netcdf`. É necessário identificar qual o formato do seu arquivo de saída. `netcdf` é possível a partir da versão _v11-02a_.

__bpch__ é o formato _default_ do Geos-Chem, embora sera retirado na versão 12.7 sendo substituido pelo `netcdf`.
Para abrir o `bpch` é preciso informar os arquivos __.dat__ que contém os nomes dos rastreadores e alguns metadados necessários.

__netcdf__, diferente do `bpch`, é um formato completo e robusto sendo auto suficiente para apresentar qualquer informação, no mesmo arquivo existem os metadados e os nomes dos rasteradores. Desse modo, para o `netcdf`, basta ele próprio.

Devido estarmos utilizando o xarray e o xbpch (interface para o xarray), o único comando que diferencia a forma de trabalho com esses arquivos é a própria abertura. Todos os próximos comandos mantem a mesma estrutura independente do formato.

### Abertura e visualização dos dados:

In [6]:
# Open all bpch ND49 (concat)
nd49 = [f for f in glob.glob(ARGS['trac'] + "ts*.bpch")]
df_trac = xbpch.open_mfbpchdataset(
    nd49,
    dask=True,
    memmap=True,
    tracerinfo_file=ARGS["tracerinfo"],
    diaginfo_file=ARGS["diaginfo"]
)
df_trac = df_trac.sortby("time")

# Open bpch
# df_trac = xbpch.open_bpchdataset(
#     ARGS["trac"],
#     tracerinfo_file=ARGS["tracerinfo"],
#     diaginfo_file=ARGS["diaginfo"],
# )

# Open netcdf
# df_trac = xr.open_dataset(ARGS["trac"])

df_trac

<xarray.Dataset>
Dimensions:      (lat: 46, lev: 72, lon: 72, nv: 2, time: 708)
Coordinates:
  * lev          (lev) float64 0.9925 0.9775 0.9624 ... 1.614e-05 4.935e-06
  * nv           (nv) int64 0 1
  * lat          (lat) float64 -89.0 -86.0 -82.0 -78.0 ... 78.0 82.0 86.0 89.0
  * lon          (lon) float64 -180.0 -175.0 -170.0 -165.0 ... 165.0 170.0 175.0
  * time         (time) datetime64[ns] 2015-01-01T02:00:00 ... 2015-03-01
Data variables:
    IJ_AVG_S_NO  (time, lon, lat, lev) float32 dask.array<chunksize=(1, 72, 46, 72), meta=np.ndarray>
    time_bnds    (time, nv) datetime64[ns] 2015-01-01T02:00:00 ... 2015-03-01
    IJ_AVG_S_O3  (time, lon, lat, lev) float32 dask.array<chunksize=(1, 72, 46, 72), meta=np.ndarray>
    IJ_AVG_S_CO  (time, lon, lat, lev) float32 dask.array<chunksize=(1, 72, 46, 72), meta=np.ndarray>
Attributes:
    modelname:    GEOSFP
    halfpolar:    1
    center180:    1
    res:          (5.0, 4.0)
    Conventions:  CF1.6
    source:       /home/user/dev/my

Listar todas variáveis existentes no arquivo:

In [7]:
variables = list(df_trac.data_vars)
variables.sort() # ordena
variables

['IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O3', 'time_bnds']

É possível selecionar uma variável específica por manipulação do `df_trac`:

In [8]:
@interact(variable=widgets.Dropdown(options=variables))
def _show(variable):
    try:
        display(df_trac[variable])
    except Exception as e:
        display(e)

interactive(children=(Dropdown(description='variable', options=('IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O3', '…

Existem duas variáveis que podem ser descartadas por não serem muito úteis:

- `time_bnds`: limites de tempo
- `nv`: dimensão redundante

Exibição dos `time_bnds`:

In [9]:
try:
    df_trac["time_bnds"]
except Exception as e:
    display(e)

Exibição da `nv`:

In [10]:
try:
    df_trac["nv"]
except Exception as e:
    display(e)

Descarte:

In [11]:
try:
    df_trac = df_trac.drop("time_bnds")
    df_trac = df_trac.drop("nv")
except Exception as e:
    display(e)

Conforme citado, existem três tipos de componentes na estrutura: as variáveis, coordenadas e atributos. Anteriormente foi explicado como é possível obter a lista de variáveis presentes no arquivo, abaixo será abordado como obter as coordenadas e atributos.

Coordenadas existentes em CO:

In [12]:
@interact(variable=widgets.Dropdown(options=variables))
def _show(variable):
    try:
        display(df_trac[variable].coords)
    except Exception as e:
        display(e)

interactive(children=(Dropdown(description='variable', options=('IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O3', '…

É possível também obter a lista através de `dims`

In [13]:
@interact(variable=widgets.Dropdown(options=variables))
def _show(variable):
    try:
        display(df_trac[variable].dims)
    except Exception as e:
        display(e)

interactive(children=(Dropdown(description='variable', options=('IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O3', '…

Atributos existentes em CO:

In [14]:
@interact(variable=widgets.Dropdown(options=variables))
def _show(variable):
    try:
        display(df_trac[variable].attrs)
    except Exception as e:
        display(e)

interactive(children=(Dropdown(description='variable', options=('IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O3', '…

O comando `values` possibilita o acesso a todos os valores existentes da matriz multidimensional (`df_trac["IJ_AVG_S_CO"].values`). Porém, recomendo o uso de `sel` ou `isel` que permite o acesso as n-dimensões semelhante ao `loc` e `iloc` do numpy (`df_trac["IJ_AVG_S_CO"].isel(time=0, lev=0)`).

Mais informações em [indexing with xarray](http://xarray.pydata.org/en/stable/indexing.html).

### Manipulação do arquivo

Algumas variáveis possuem dimensão 4D. A ordem convencional é (time, lev, lat, lon), no `bpch` é observado a ordem (time, lon, lat, lev). Com a função `transpose` é possível alterar a ordem da dimensionalidade.

_Obs: verifique se na sua entrada não existem variáveis com dimensões divergentes, caso exista, antes de executar o comando abaixo é necessário que extraia apenas as variáveis que respeitem a dimensão com as coordenadas informadas. Uma forma de extrair apenas essas variáveis pode ser feitas através do comando:_

- `df_extracao = df_trac[[lista_de_variáveis_de_interesse]]`

_Exemplo:_ Obtendo os dados das concentrações de __O3__ e __SO2__

- `df_extracao = df_trac[['IJ_AVG_S_O3', 'IJ_AVG_S_SO2']]`

_Desse modo, as manipulações podem ser feitas sobre a variável df_extracao_

_Obs: O formato bpch inverte a dimensionalidade entre lat e lon, prejudicando a exibição real. `transpose` pode corrigir esse problema._

In [15]:
# df_trac = df_trac.transpose("time", "lev", "lat", "lon")
df_trac = df_trac.transpose()

É possível também converter o arquivo `bpch` para `netcdf`:

In [16]:
# df_trac_fe = xbpch.common.fix_attr_encoding(df_trac)
# df_trac_fe.to_netcdf(f"{ARGS['trac']}.nc")

Caso possua o `ncdump` instalado, é possível executar o comando do terminal de leitura do arquivo:

O caracter __!__ indica a o jupyter que o comando deve ser executado no terminal.

In [17]:
# !ncdump -h caminho_do_trac.nc

### Plot

Esse notebook foi desenvolvido com objetivo de possibilitar uma análise gráfica estática e dinâmica. A função `plot_static`, apresentada no inicio do documento, possibilita selecionar as variáveis, períodos e lev específicos para plotagem e salvar seu output. Enquanto que a função `plot_dynamic`, apresentado abaixo, possibilita uma visualização interativa.

#### Static

In [18]:
@interact(tracers=widgets.SelectMultiple(options=variables))
def _plot(tracers):
    try:
        plot_static(
            emiss=df_trac,
            tracers=list(tracers),
            times=[0, 1],
            levs=[0, 22],
            save=True
        )
    except Exception as e:
        display(e)

interactive(children=(SelectMultiple(description='tracers', options=('IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O…

#### Interactive

In [19]:
@interact(
    variable=widgets.Dropdown(options=variables, continuous_update=False), 
    lev=widgets.IntSlider(min=0, max=46, step=1, continuous_update=False),
)
def plot_dynamic(variable, lev):
    try:
        # Configures plot
        tracer = df_trac[variable]
        fig = plt.figure(figsize=(14, 7))
        ax = plt.axes(projection=ccrs.PlateCarree())
        kwargs = {}
        
        # Initial time
        if "time" in tracer.dims:
            kwargs["time"] = 0
            tracer_t = tracer.time
            time_v = tracer_t.values[0]
        else:
            tracer_t = [0]
            time_v = "there is no time"
        
        # Checks if exist lev
        if "lev" in tracer.dims:
            kwargs["lev"] = lev
            lev_v = tracer.lev.values[lev]
        else:
            lev_v = "there is no lev"
        
        # Initial plot
        image = tracer.isel(**kwargs).plot.imshow(
            ax=ax, cmap=WhGrYlRd, animated=True
        )
        
        # Reconfigures plot
        ax.set_title(f"lev = {lev_v} | time = {time_v}", y=1.05)
        ax.gridlines(draw_labels=True, linestyle="--")
        ax.coastlines()
        
        def update(t):
            # Update time in kwargs and get full time
            if "time" in tracer.dims:
                kwargs["time"] = t
                time_v = tracer_t.values[t]
            else:
                time_v = "there is no time"
            
            
            # Update the plot for a specific time
            ax.set_title(f"lev = {lev_v} | time = {time_v}")
            image.set_array(tracer.isel(**kwargs))
            return image,

        # Run the animation, applying `update()` for each of the times in the tracer
        anim = FuncAnimation(
            fig,
            update,
            frames=range(len(tracer_t)),
            blit=True,
            interval=1000
        )
        
        anim.save(ARGS['anim'], writer=writer)
        plt.close(anim._fig)
        return HTML(anim.to_html5_video())
    except Exception as e:
        print(f"{e}")

interactive(children=(Dropdown(description='variable', options=('IJ_AVG_S_CO', 'IJ_AVG_S_NO', 'IJ_AVG_S_O3', '…

Fonte: [GEOSChem-python-tutorial](https://github.com/geoschem/GEOSChem-python-tutorial)